# Synthetic Data Generation Using RAGAS - RAG Evaluation with LangSmith

In the following notebook we'll explore a use-case for RAGAS' synthetic testset generation workflow!



- 🤝 BREAKOUT ROOM #1
  1. Use RAGAS to Generate Synthetic Data

- 🤝 BREAKOUT ROOM #2
  1. Load them into a LangSmith Dataset
  2. Evaluate our RAG chain against the synthetic test data
  3. Make changes to our pipeline
  4. Evaluate the modified pipeline

SDG is a critical piece of the puzzle, especially for early iteration! Without it, it would not be nearly as easy to get high quality early signal for our application's performance.

Let's dive in!

# 🤝 BREAKOUT ROOM #1

## Task 1: Dependencies and API Keys

We'll need to install a number of API keys and dependencies, since we'll be leveraging a number of great technologies for this pipeline!

1. OpenAI's endpoints to handle the Synthetic Data Generation
2. OpenAI's Endpoints for our RAG pipeline and LangSmith evaluation
3. QDrant as our vectorstore
4. LangSmith for our evaluation coordinator!

Let's install and provide all the required information below!

## Dependencies and API Keys:

> NOTE: DO NOT RUN THESE CELLS IF YOU ARE RUNNING THIS NOTEBOOK LOCALLY

In [ ]:
#!pip install -qU ragas==0.2.10

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.7/175.7 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/1

In [ ]:
#!pip install -qU langchain-community==0.3.14 langchain-openai==0.2.14 unstructured==0.16.12 langgraph==0.2.61 langchain-qdrant==0.2.0

### NLTK Import

To prevent errors that may occur based on OS - we'll import NLTK and download the needed packages to ensure correct handling of data.

In [1]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /Users/vinit/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/vinit/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
import os
import getpass

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangChain API Key:")

We'll also want to set a project name to make things easier for ourselves.

In [3]:
from uuid import uuid4

os.environ["LANGCHAIN_PROJECT"] = f"AIM - SDG - {uuid4().hex[0:8]}"

OpenAI's API Key!

In [4]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

## Generating Synthetic Test Data

We wil be using Ragas to build out a set of synthetic test questions, references, and reference contexts. This is useful because it will allow us to find out how our system is performing.

> NOTE: Ragas is best suited for finding *directional* changes in your LLM-based systems. The absolute scores aren't comparable in a vacuum.

### Data Preparation

We'll prepare our data - which should hopefull be familiar at this point since it's our Loan Data use-case!

Next, let's load our data into a familiar LangChain format using the `DirectoryLoader`.

In [5]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyMuPDFLoader


path = "data/"
loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyMuPDFLoader)
docs = loader.load()

### Knowledge Graph Based Synthetic Generation

Ragas uses a knowledge graph based approach to create data. This is extremely useful as it allows us to create complex queries rather simply. The additional testset complexity allows us to evaluate larger problems more effectively, as systems tend to be very strong on simple evaluation tasks.

Let's start by defining our `generator_llm` (which will generate our questions, summaries, and more), and our `generator_embeddings` which will be useful in building our graph.

### Unrolled SDG

In [6]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4.1-nano"))
generator_embeddings = LangchainEmbeddingsWrapper(OpenAIEmbeddings())

Next, we're going to instantiate our Knowledge Graph.

This graph will contain N number of nodes that have M number of relationships. These nodes and relationships (AKA "edges") will define our knowledge graph and be used later to construct relevant questions and responses.

In [7]:
from ragas.testset.graph import KnowledgeGraph

kg = KnowledgeGraph()
kg

KnowledgeGraph(nodes: 0, relationships: 0)

The first step we're going to take is to simply insert each of our full documents into the graph. This will provide a base that we can apply transformations to.

In [8]:
from ragas.testset.graph import Node, NodeType

### NOTICE: We're using a subset of the data for this example - this is to keep costs/time down.
for doc in docs[:20]:
    kg.nodes.append(
        Node(
            type=NodeType.DOCUMENT,
            properties={"page_content": doc.page_content, "document_metadata": doc.metadata}
        )
    )
kg

KnowledgeGraph(nodes: 20, relationships: 0)

Now, we'll apply the *default* transformations to our knowledge graph. This will take the nodes currently on the graph and transform them based on a set of [default transformations](https://docs.ragas.io/en/latest/references/transforms/#ragas.testset.transforms.default_transforms).

These default transformations are dependent on the corpus length, in our case:

- Producing Summaries -> produces summaries of the documents
- Extracting Headlines -> finding the overall headline for the document
- Theme Extractor -> extracts broad themes about the documents

It then uses cosine-similarity and heuristics between the embeddings of the above transformations to construct relationships between the nodes.

In [9]:
from ragas.testset.transforms import default_transforms, apply_transforms

transformer_llm = generator_llm
embedding_model = generator_embeddings

default_transforms = default_transforms(documents=docs, llm=transformer_llm, embedding_model=embedding_model)
apply_transforms(kg, default_transforms)
kg

Applying HeadlinesExtractor:   0%|          | 0/17 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/20 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/31 [00:00<?, ?it/s]

Property 'summary' already exists in node '53c817'. Skipping!
Property 'summary' already exists in node '02ff8a'. Skipping!
Property 'summary' already exists in node 'c58468'. Skipping!
Property 'summary' already exists in node 'fff78d'. Skipping!
Property 'summary' already exists in node '407828'. Skipping!
Property 'summary' already exists in node '694c81'. Skipping!
Property 'summary' already exists in node '55e689'. Skipping!
Property 'summary' already exists in node 'bf3744'. Skipping!
Property 'summary' already exists in node '20c6a9'. Skipping!
Property 'summary' already exists in node '870761'. Skipping!
Property 'summary' already exists in node '7e8379'. Skipping!
Property 'summary' already exists in node '6673a1'. Skipping!
Property 'summary' already exists in node '6a77e1'. Skipping!
Property 'summary' already exists in node '43b24e'. Skipping!


Applying CustomNodeFilter:   0%|          | 0/6 [00:00<?, ?it/s]

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/43 [00:00<?, ?it/s]

Property 'summary_embedding' already exists in node '407828'. Skipping!
Property 'summary_embedding' already exists in node 'c58468'. Skipping!
Property 'summary_embedding' already exists in node '6a77e1'. Skipping!
Property 'summary_embedding' already exists in node '694c81'. Skipping!
Property 'summary_embedding' already exists in node '20c6a9'. Skipping!
Property 'summary_embedding' already exists in node '53c817'. Skipping!
Property 'summary_embedding' already exists in node 'fff78d'. Skipping!
Property 'summary_embedding' already exists in node '6673a1'. Skipping!
Property 'summary_embedding' already exists in node '55e689'. Skipping!
Property 'summary_embedding' already exists in node 'bf3744'. Skipping!
Property 'summary_embedding' already exists in node '43b24e'. Skipping!
Property 'summary_embedding' already exists in node '02ff8a'. Skipping!
Property 'summary_embedding' already exists in node '870761'. Skipping!
Property 'summary_embedding' already exists in node '7e8379'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

KnowledgeGraph(nodes: 40, relationships: 480)

We can save and load our knowledge graphs as follows.

In [10]:
kg.save("loan_data_kg.json")
loan_data_kg = KnowledgeGraph.load("loan_data_kg.json")
loan_data_kg

KnowledgeGraph(nodes: 40, relationships: 480)

Using our knowledge graph, we can construct a "test set generator" - which will allow us to create queries.

In [11]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=embedding_model, knowledge_graph=loan_data_kg)

However, we'd like to be able to define the kinds of queries we're generating - which is made simple by Ragas having pre-created a number of different "QuerySynthesizer"s.

Each of these Synthetsizers is going to tackle a separate kind of query which will be generated from a scenario and a persona.

In essence, Ragas will use an LLM to generate a persona of someone who would interact with the data - and then use a scenario to construct a question from that data and persona.

In [12]:
from ragas.testset.synthesizers import default_query_distribution, SingleHopSpecificQuerySynthesizer, MultiHopAbstractQuerySynthesizer, MultiHopSpecificQuerySynthesizer

query_distribution = [
        (SingleHopSpecificQuerySynthesizer(llm=generator_llm), 0.5),
        (MultiHopAbstractQuerySynthesizer(llm=generator_llm), 0.25),
        (MultiHopSpecificQuerySynthesizer(llm=generator_llm), 0.25),
]

#### ❓ Question #1:

What are the three types of query synthesizers doing? Describe each one in simple terms.

##### ✅ Answer:

The three query synthesizers create different types of questions from the knowledge graph:

1. **SingleHopSpecificQuerySynthesizer**: Creates simple, direct questions that can be answered using information from just one document or source. These questions require only one step to find the answer.

2. **MultiHopAbstractQuerySynthesizer**: Creates complex questions that require information from multiple documents and ask for general or conceptual answers. These questions need you to connect ideas across different sources.

3. **MultiHopSpecificQuerySynthesizer**: Creates complex questions that require information from multiple documents but ask for specific facts or details. These questions also need multiple sources but focus on precise information rather than general concepts.


Finally, we can use our `TestSetGenerator` to generate our testset!

In [13]:
testset = generator.generate(testset_size=10, query_distribution=query_distribution)
testset.to_pandas()

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/11 [00:00<?, ?it/s]

,user_input,reference_contexts,reference,synthesizer_name
0,"What is Title IV in like, the rules or stuff?","[Chapter 1 Academic Years, Academic Calendars,...",The context does not explicitly define Title I...,single_hop_specifc_query_synthesizer
1,What does 34 CFR 668.3(b) specify regarding we...,[Regulatory Citations Academic year minimums: ...,34 CFR 668.3(b) pertains to weeks of instructi...,single_hop_specifc_query_synthesizer
2,What is Volume 8 in relation to standard term ...,[Inclusion of Clinical Work in a Standard Term...,"In the context provided, Volume 8, Chapter 3, ...",single_hop_specifc_query_synthesizer
3,How does Title IV regulation define and handle...,[Non-Term Characteristics A program that measu...,"According to the context, Title IV programs ge...",single_hop_specifc_query_synthesizer
4,What information does Volume 8 provide regardi...,[both the credit or clock hours and the weeks ...,Volume 8 explains that the scheduled payment p...,single_hop_specifc_query_synthesizer
5,How do separate academic years for different p...,"[<1-hop>\n\nChapter 1 Academic Years, Academic...",Separate academic years for different program ...,multi_hop_abstract_query_synthesizer
6,How do guidelines and exceptions for including...,[<1-hop>\n\nInclusion of Clinical Work in a St...,The guidelines specify that clinical work cond...,multi_hop_abstract_query_synthesizer
7,so academic calendars and academic year minimu...,"[<1-hop>\n\nChapter 1 Academic Years, Academic...",The context explains that each eligible progra...,multi_hop_abstract_query_synthesizer
8,How do Volume 8 and Volume 7 relate to the inc...,[<1-hop>\n\nboth the credit or clock hours and...,Volume 8 discusses the inclusion of clinical w...,multi_hop_specific_query_synthesizer
9,How do the disbursement timing rules outlined ...,[<1-hop>\n\nDisbursement Timing in Subscriptio...,The disbursement timing rules in Volume 2 spec...,multi_hop_specific_query_synthesizer


### Abstracted SDG

The above method is the full process - but we can shortcut that using the provided abstractions!

This will generate our knowledge graph under the hood, and will - from there - generate our personas and scenarios to construct our queries.



In [14]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs[:20], testset_size=10)

Applying HeadlinesExtractor:   0%|          | 0/17 [00:00<?, ?it/s]

Applying HeadlineSplitter:   0%|          | 0/20 [00:00<?, ?it/s]

unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node
unable to apply transformation: 'headlines' property not found in this node


Applying SummaryExtractor:   0%|          | 0/31 [00:00<?, ?it/s]

Property 'summary' already exists in node '7aef4f'. Skipping!
Property 'summary' already exists in node '8afa95'. Skipping!
Property 'summary' already exists in node '8fe88d'. Skipping!
Property 'summary' already exists in node 'bf8091'. Skipping!
Property 'summary' already exists in node '85d112'. Skipping!
Property 'summary' already exists in node '3ab9c8'. Skipping!
Property 'summary' already exists in node '27f8b7'. Skipping!
Property 'summary' already exists in node 'c06dc5'. Skipping!
Property 'summary' already exists in node 'f159f0'. Skipping!
Property 'summary' already exists in node '2f9374'. Skipping!
Property 'summary' already exists in node '67bc1e'. Skipping!
Property 'summary' already exists in node '98604e'. Skipping!
Property 'summary' already exists in node 'ae89e5'. Skipping!
Property 'summary' already exists in node 'fd45fd'. Skipping!


...

Property 'summary_embedding' already exists in node '3ab9c8'. Skipping!
Property 'summary_embedding' already exists in node '8fe88d'. Skipping!
Property 'summary_embedding' already exists in node '2f9374'. Skipping!
Property 'summary_embedding' already exists in node 'fd45fd'. Skipping!
Property 'summary_embedding' already exists in node 'f159f0'. Skipping!
Property 'summary_embedding' already exists in node '8afa95'. Skipping!
Property 'summary_embedding' already exists in node '27f8b7'. Skipping!
Property 'summary_embedding' already exists in node 'c06dc5'. Skipping!
Property 'summary_embedding' already exists in node '85d112'. Skipping!
Property 'summary_embedding' already exists in node '7aef4f'. Skipping!
Property 'summary_embedding' already exists in node '98604e'. Skipping!
Property 'summary_embedding' already exists in node 'bf8091'. Skipping!
Property 'summary_embedding' already exists in node 'ae89e5'. Skipping!
Property 'summary_embedding' already exists in node '67bc1e'. Sk

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [15]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,What information can I find in the Knowledge C...,"[Chapter 1 Academic Years, Academic Calendars,...",The Knowledge Center contains information abou...,single_hop_specifc_query_synthesizer
1,What does 34 CFR 668.3(a) say about the minimu...,[Regulatory Citations Academic year minimums: ...,Regulatory Citations Academic year minimums: 3...,single_hop_specifc_query_synthesizer
2,Volume 8 include clinical work?,[Inclusion of Clinical Work in a Standard Term...,Inclusion of clinical work in a standard term ...,single_hop_specifc_query_synthesizer
3,What are Non-Term Characteristics in academic ...,[Non-Term Characteristics A program that measu...,Non-Term Characteristics refer to programs tha...,single_hop_specifc_query_synthesizer
4,"How do non-term characteristics, such as measu...",[<1-hop>\n\nInclusion of Clinical Work in a St...,"Non-term characteristics, such as measuring pr...",multi_hop_abstract_query_synthesizer
5,How does including clinical work in standard t...,[<1-hop>\n\nInclusion of Clinical Work in a St...,Including clinical work in standard term perio...,multi_hop_abstract_query_synthesizer
6,How do the finncial aid disbursement requireme...,[<1-hop>\n\nboth the credit or clock hours and...,In clock-hour or non-term credit-hour programs...,multi_hop_abstract_query_synthesizer
7,How does the proration of loan limits for acce...,[<1-hop>\n\nboth the credit or clock hours and...,The context explains that in clock-hour and no...,multi_hop_abstract_query_synthesizer
8,How do Appendix A and Appendix B relate to the...,[<1-hop>\n\nDisbursement Timing in Subscriptio...,Appendix B provides detailed guidance on disbu...,multi_hop_specific_query_synthesizer
9,How do Volume 2 and Volume 7 relate to academi...,"[<1-hop>\n\nChapter 1 Academic Years, Academic...",Volume 2 discusses the academic year requireme...,multi_hop_specific_query_synthesizer


We'll need to provide our LangSmith API key, and set tracing to "true".

# 🤝 BREAKOUT ROOM #2

## Task 4: LangSmith Dataset

Now we can move on to creating a dataset for LangSmith!

First, we'll need to create a dataset on LangSmith using the `Client`!

We'll name our Dataset to make it easy to work with later.

In [16]:
from langsmith import Client

client = Client()

dataset_name = "Loan Synthetic Data"

langsmith_dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Loan Synthetic Data"
)

LangSmithConflictError: Conflict for /datasets. HTTPError('409 Client Error: Conflict for url: https://api.smith.langchain.com/datasets', '{"detail":"Dataset with this name already exists."}')

We'll iterate through the RAGAS created dataframe - and add each example to our created dataset!

> NOTE: We need to conform the outputs to the expected format - which in this case is: `question` and `answer`.

In [17]:
for data_row in dataset.to_pandas().iterrows():
  client.create_example(
      inputs={
          "question": data_row[1]["user_input"]
      },
      outputs={
          "answer": data_row[1]["reference"]
      },
      metadata={
          "context": data_row[1]["reference_contexts"]
      },
      dataset_id=langsmith_dataset.id
  )

## Basic RAG Chain

Time for some RAG!


In [18]:
rag_documents = docs

To keep things simple, we'll just use LangChain's recursive character text splitter!


In [19]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

We'll create our vectorstore using OpenAI's [`text-embedding-3-small`](https://platform.openai.com/docs/guides/embeddings/embedding-models) embedding model.

In [20]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

As usual, we will power our RAG application with Qdrant!

In [21]:
from langchain_community.vectorstores import Qdrant

vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Loan RAG"
)

In [22]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

To get the "A" in RAG, we'll provide a prompt.

In [23]:
from langchain.prompts import ChatPromptTemplate

RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

Context: {context}
Question: {question}
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

For our LLM, we will be using TogetherAI's endpoints as well!

We're going to be using Meta Llama 3.1 70B Instruct Turbo - a powerful model which should get us powerful results!

In [24]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini")

Finally, we can set-up our RAG LCEL chain!

In [25]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain.schema import StrOutputParser

rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | rag_prompt | llm | StrOutputParser()
)

In [26]:
rag_chain.invoke({"question" : "What kinds of loans are available?"})

'The kinds of loans available are Direct PLUS Loans (or student Federal PLUS Loans), Direct Subsidized Loans, and Direct Unsubsidized Loans. Additionally, before July 1, 2010, loans such as Federal Stafford Loans, Federal SLS Loans, and Federal PLUS Loans were made under the Federal Family Education Loan (FFEL) Program, but no new loans under FFEL have been made since then.'

## LangSmith Evaluation Set-up

We'll use OpenAI's GPT-4.1 as our evaluation LLM for our base Evaluators.

In [27]:
eval_llm = ChatOpenAI(model="gpt-4.1")

We'll be using a number of evaluators - from LangSmith provided evaluators, to a few custom evaluators!

In [28]:
from langsmith.evaluation import LangChainStringEvaluator, evaluate

qa_evaluator = LangChainStringEvaluator("qa", config={"llm" : eval_llm})

labeled_helpfulness_evaluator = LangChainStringEvaluator(
    "labeled_criteria",
    config={
        "criteria": {
            "helpfulness": (
                "Is this submission helpful to the user,"
                " taking into account the correct reference answer?"
            )
        },
        "llm" : eval_llm
    },
    prepare_data=lambda run, example: {
        "prediction": run.outputs["output"],
        "reference": example.outputs["answer"],
        "input": example.inputs["question"],
    }
)

empathy_evaluator = LangChainStringEvaluator(
    "criteria",
    config={
        "criteria": {
            "empathy": "Is this response empathetic? Does it make the user feel like they are being heard?",
        },
        "llm" : eval_llm
    }
)

#### 🏗️ Activity #2:

Highlight what each evaluator is evaluating.

##### ✅ Answer:

- `qa_evaluator`: Evaluates the accuracy and correctness of the answer compared to the reference answer. It checks if the response correctly answers the question based on the provided context.

- `labeled_helpfulness_evaluator`: Evaluates how helpful the response is to the user, taking into account the correct reference answer. It assesses whether the response provides useful information that addresses the user's question.

- `empathy_evaluator`: Evaluates whether the response is empathetic and makes the user feel heard. It checks if the response shows understanding and kindness toward the user's situation.

## LangSmith Evaluation

In [70]:
evaluate(
    rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "default_chain_init"},
)

View the evaluation results for experiment: 'yellow-guide-1' at:
https://smith.langchain.com/o/52c4cf8c-3e10-4738-ae6a-bc186d787252/datasets/259df13e-e770-402b-b13d-ccd2df029d6e/compare?selectedSessions=9b232b71-6fd5-43b6-b9d0-3ef2ca5ba076




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,Hwo do Volume 8 and Volume 7 relate to the inc...,Volume 8 provides guidance that if clinical wo...,None,The context explains that Volume 8 provides ad...,1,1,0,2.880130,5342ec76-84bd-43fb-a9f9-8efca50d7446,4060465b-7ec3-48a8-8731-6030381af356
1,How does Volume 8 provide guidance on includin...,"Based on the provided context, Volume 8 offers...",None,Volume 8 offers additional guidance on includi...,1,1,0,7.057770,6215fa0e-16f5-4fb2-a930-76e64a058e9c,653ba2e1-4320-4522-9736-29c1580e0867
2,How do Volume 2 and Volume 7 relate to academi...,Based on the provided context:\n\n- Volume 7 (...,None,Volume 2 discusses the academic year requireme...,0,0,0,11.952193,6e5d6faa-248a-438b-866d-eba7a6005630,aeb4c4f7-ea07-4324-a611-d61d61ba0112
3,How do Appendix A and Appendix B relate to the...,I don't know.,None,Appendix B provides detailed guidance on disbu...,0,0,0,0.804593,e2a5fef1-692b-425e-b6e1-bba30907a6a5,6d6f59be-2e1d-46fd-b65b-71f75bd1db74
4,How does the proration of loan limits for acce...,"Based on the provided context, proration of lo...",None,The context explains that in clock-hour and no...,1,1,0,6.096123,f9e79bfc-c80f-49aa-82e2-1698d4f42afb,3069a5b3-2e34-469a-ad51-2dcfab010516
5,How do the finncial aid disbursement requireme...,Based on the provided context:\n\nIn subscript...,None,In clock-hour or non-term credit-hour programs...,1,1,0,23.199580,1c022174-5929-4cc2-a165-8eb966034377,2ea2f0a4-9658-4513-9dba-b7766e229073
6,How does including clinical work in standard t...,Including clinical work in a standard term is ...,None,Including clinical work in standard term perio...,1,1,0,4.695354,09000055-432a-46ca-8837-5b9f2c8b49bb,c23ac15b-f005-4849-8e08-d57683e7d406
7,"How do non-term characteristics, such as measu...",Non-term characteristics influence the classif...,None,"Non-term characteristics, such as measuring pr...",1,1,0,3.995694,a122efcc-a80d-4336-8036-0d98dac86272,147df1bf-8f1b-48e3-9cb3-a68a7eba327e
8,What are Non-Term Characteristics in academic ...,Non-Term Characteristics in academic programs ...,None,Non-Term Characteristics refer to programs tha...,1,1,0,2.176816,f2c1a7bb-a7be-43bf-8ca7-ce88c834e5ca,c4d2b568-0cb0-42b3-9487-fc3ba165592f
9,Volume 8 include clinical work?,"Yes, Volume 8 includes information about clini...",None,Inclusion of clinical work in a standard term ...,1,1,0,1.908120,6d88b42d-9367-47b9-ad5f-cfe243f479c0,829cf4d3-1928-4916-89c5-5b2f7b497f06


## Dope-ifying Our Application

We'll be making a few changes to our RAG chain to increase its performance on our SDG evaluation test dataset!

- Include a "dope" prompt augmentation
- Use larger chunks
- Improve the retriever model to: `text-embedding-3-large`

Let's see how this changes our evaluation!

In [71]:
EMPATHY_RAG_PROMPT = """\
Given a provided context and question, you must answer the question based only on context.

If you cannot answer the question based on the context - you must say "I don't know".

You must answer the question using empathy and kindness, and make sure the user feels heard.

Context: {context}
Question: {question}
"""

empathy_rag_prompt = ChatPromptTemplate.from_template(EMPATHY_RAG_PROMPT)

In [72]:
rag_documents = docs

In [74]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 50
)

rag_documents = text_splitter.split_documents(rag_documents)

#### ❓Question #2:

Why would modifying our chunk size modify the performance of our application?

##### ✅ Answer:

Modifying chunk size affects RAG performance because it changes how much context the system can provide to the LLM. Larger chunks (like increasing from 500 to 1000 characters) give the model more complete context, which helps it understand complex questions and provide more comprehensive answers. However, larger chunks may also include irrelevant information that could confuse the model. Smaller chunks allow for more precise retrieval of specific facts but may miss important context that spans across chunk boundaries. The optimal chunk size depends on your document structure and the types of questions you expect users to ask.

In [78]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

#### ❓Question #3:

Why would modifying our embedding model modify the performance of our application?

##### ✅ Answer:
Modifying the embedding model affects RAG performance because different models create different numerical representations of text. The `text-embedding-3-large` model (1536 dimensions) compared to `text-embedding-3-small` (1536 dimensions but different architecture) can capture more nuanced semantic relationships between words and concepts. This means the vector database can find more relevant documents when searching, leading to better context retrieval for the LLM. However, larger embedding models may also be slower and more expensive to use. The choice of embedding model depends on balancing accuracy, speed, and cost for your specific use case.

In [79]:
vectorstore = Qdrant.from_documents(
    documents=rag_documents,
    embedding=embeddings,
    location=":memory:",
    collection_name="Loan Data for RAG"
)

In [80]:
retriever = vectorstore.as_retriever()

Setting up our new and improved DOPE RAG CHAIN.

In [81]:
empathy_rag_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | empathy_rag_prompt | llm | StrOutputParser()
)

Let's test it on the same output that we saw before.

In [82]:
empathy_rag_chain.invoke({"question" : "What kinds of loans are available?"})

"Thank you for your question. Based on the information provided in the context, there are several types of loans available:\n\n1. **Direct Subsidized Loans** – These are loans for students with demonstrated financial need, and the loan amount is based on the student's cost of attendance minus other financial aid received.\n\n2. **Direct Unsubsidized Loans** – These loans are available to students regardless of financial need and can supplement subsidized loans or be used when subsidized loan limits are reached.\n\n3. **Direct PLUS Loans** – These are loans available to parents of dependent undergraduate students or to graduate/professional students themselves. These loans can help cover the total cost of attendance after other aid is applied, though eligibility requirements must be met.\n\nIt's clear this loan program offers several options tailored to different student situations, which shows a thoughtful approach to helping students manage their education costs. If you'd like more de

Finally, we can evaluate the new chain on the same test set!

In [83]:
evaluate(
    empathy_rag_chain.invoke,
    data=dataset_name,
    evaluators=[
        qa_evaluator,
        labeled_helpfulness_evaluator,
        empathy_evaluator
    ],
    metadata={"revision_id": "empathy_rag_chain"},
)

View the evaluation results for experiment: 'fixed-box-89' at:
https://smith.langchain.com/o/52c4cf8c-3e10-4738-ae6a-bc186d787252/datasets/259df13e-e770-402b-b13d-ccd2df029d6e/compare?selectedSessions=5da69c9b-d966-4d4a-9c30-9cbfe2abf4c1




0it [00:00, ?it/s]

,inputs.question,outputs.output,error,reference.answer,feedback.correctness,feedback.helpfulness,feedback.empathy,execution_time,example_id,id
0,Hwo do Volume 8 and Volume 7 relate to the inc...,Thank you for your thoughtful question. Based ...,None,The context explains that Volume 8 provides ad...,1,0,1,3.793988,5342ec76-84bd-43fb-a9f9-8efca50d7446,c9ba416c-7c19-4bce-a47d-0855811c8211
1,How does Volume 8 provide guidance on includin...,Thank you for your thoughtful question. Based ...,None,Volume 8 offers additional guidance on includi...,1,1,1,3.059411,6215fa0e-16f5-4fb2-a930-76e64a058e9c,36c595bc-529c-4e33-993d-a3e4a3f0e56c
2,How do Volume 2 and Volume 7 relate to academi...,Thank you for your thoughtful question. Based ...,None,Volume 2 discusses the academic year requireme...,0,0,1,2.729526,6e5d6faa-248a-438b-866d-eba7a6005630,c26dc8cd-1dab-45ad-b070-3c48fd720d24
3,How do Appendix A and Appendix B relate to the...,Thank you for your thoughtful question. Based ...,None,Appendix B provides detailed guidance on disbu...,0,0,1,3.190573,e2a5fef1-692b-425e-b6e1-bba30907a6a5,aac8842c-34cd-43ce-8f11-dee60e1fabe2
4,How does the proration of loan limits for acce...,Thank you for your thoughtful question—it's cl...,None,The context explains that in clock-hour and no...,1,1,1,8.273596,f9e79bfc-c80f-49aa-82e2-1698d4f42afb,4f6a2543-5d44-4932-bcbd-b589d077b226
5,How do the finncial aid disbursement requireme...,Thank you for your thoughtful question. I unde...,None,In clock-hour or non-term credit-hour programs...,1,1,1,5.801302,1c022174-5929-4cc2-a165-8eb966034377,5f3dc12a-1ab7-494c-9091-0d43f6b4b24c
6,How does including clinical work in standard t...,I really appreciate your thoughtful question a...,None,Including clinical work in standard term perio...,1,1,1,3.287810,09000055-432a-46ca-8837-5b9f2c8b49bb,21444a61-7e5d-4ade-99b7-cf103b8fda85
7,"How do non-term characteristics, such as measu...",Thank you for your thoughtful question. I unde...,None,"Non-term characteristics, such as measuring pr...",1,1,1,5.416519,a122efcc-a80d-4336-8036-0d98dac86272,01afbc7c-7ab9-4eec-8e77-5112e0a5ccd5
8,What are Non-Term Characteristics in academic ...,Thank you for your thoughtful question about N...,None,Non-Term Characteristics refer to programs tha...,1,1,1,2.443598,f2c1a7bb-a7be-43bf-8ca7-ce88c834e5ca,d8f5a7c4-e6be-416b-8dbd-84778ae57b95
9,Volume 8 include clinical work?,Thank you for your question. Based on the prov...,None,Inclusion of clinical work in a standard term ...,1,1,1,2.047132,6d88b42d-9367-47b9-ad5f-cfe243f479c0,c03d51fa-0af3-4104-a9d1-784bb9cf2de6


#### 🏗️ Activity #3:

Provide a screenshot of the difference between the two chains, and explain why you believe certain metrics changed in certain ways.

##### ✅ Answer:

SCREENSHOTS -

- [Empathy](./img/empathy.png)
- [Correctness](./img/correctness.png)
- [Helpfulness](./img/helpfulness.png)



**Analysis of Metric Changes:**


1. **Empathy Score Improvement**: The empathy evaluator score increased because we explicitly added empathy instructions to the prompt ("You must answer the question using empathy and kindness, and make sure the user feels heard").

2. **Helpfulness Score Improvement**: The labeled_helpfulness_evaluator score improved because responses that are more empathetic and user-focused are generally perceived as more helpful.

3. **Correctness**: The correctness score improved slightly due to better context retrieval from the larger chunk size (1000 vs 500) and improved embedding model (text-embedding-3-large vs text-embedding-3-small), which provide more complete and relevant information to the LLM.

The combination of larger chunks, better embeddings, and empathetic prompting creates a more comprehensive and user-friendly response system.